In [4]:
import os
import sys
import h5py
import json
import numpy as np
import torch as pt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

import pickle
import lzma

# from src.dataset import StructuresDataset, collate_batch_features, select_by_sid, select_by_interface_types
from src.data_encoding import encode_res, all_resnames, selected_locations, encode_location
from config import config_data, config_runtime
from model import RNN
from data_handler import FastaDataset
from torch.utils.data import DataLoader
# from src.structure import data_to_structure, encode_bfactor, concatenate_chains, split_by_chain
# from src.structure_io import save_pdb, read_pdb
# from src.scoring import bc_scoring, bc_score_names

In [5]:
with lzma.open("../data/data_seq_locations.xz", "r") as f:
    combined_data = pickle.load(f)
   

In [6]:
enc_loc_list = []
for i, l in enumerate(combined_data['loc']):
    e_l = encode_location(l)
    enc_loc_list.append(e_l)
enc_loc_np = np.array(enc_loc_list)

In [7]:
print(np.sum(enc_loc_np, axis =0))
print(selected_locations)


[32348  5633 12110  6942  5633]
['membrane', 'cytoplasm', 'mitochondrion', 'nucleus', 'cytoplasm']


In [8]:
# data parameters
nres = 1024
data_path = "../examples/"


In [ ]:
%ls "../examples/"

In [9]:
dast_ds = FastaDataset(data_path, nres)
# fast_dl = DataLoader(dast_ds, batch_size=dast_ds.__len__())

In [10]:
# define 
device = pt.device("cpu")


n_letters = len(all_resnames)
n_categories = len(selected_locations)
learning_rate = config_runtime['learning_rate']
n_hidden = config_runtime['hidden_size']
n_layers = config_runtime['layers']

# create model
model = RNN(nres, n_hidden, n_layers, n_categories, device)
# reload model
model.load_state_dict(pt.load("model.pt", map_location=pt.device("cpu")))
# set model to inference
model = model.eval().to(device)

In [ ]:
# dast_ds.getall()

In [11]:
ids = dast_ds.getall()[0]
sm = pt.nn.Softmax(dim=1)
results = sm(model(dast_ds.getall()[1]))

In [12]:
for i, ii in enumerate(ids):
    print()
    print(f'===== {ii} ====')
    j = results[i].detach().numpy()
    for il, l in enumerate(selected_locations):
        print(f'{l}, p={j[il]:.2f}')
        


===== CCL20_MOUSE ====
membrane, p=0.42
cytoplasm, p=0.18
mitochondrion, p=0.09
nucleus, p=0.13
cytoplasm, p=0.17

===== GID4_MOUSE ====
membrane, p=0.77
cytoplasm, p=0.07
mitochondrion, p=0.06
nucleus, p=0.04
cytoplasm, p=0.07

===== XCL1_RAT ====
membrane, p=0.31
cytoplasm, p=0.21
mitochondrion, p=0.03
nucleus, p=0.23
cytoplasm, p=0.21

===== NUP62_MOUSE ====
membrane, p=0.04
cytoplasm, p=0.19
mitochondrion, p=0.01
nucleus, p=0.56
cytoplasm, p=0.20

===== MCL1_MOUSE ====
membrane, p=0.74
cytoplasm, p=0.08
mitochondrion, p=0.05
nucleus, p=0.04
cytoplasm, p=0.08

===== FTSZ_BACSU ====
membrane, p=0.42
cytoplasm, p=0.20
mitochondrion, p=0.07
nucleus, p=0.12
cytoplasm, p=0.19

===== MIM2_SCHPO ====
membrane, p=0.22
cytoplasm, p=0.24
mitochondrion, p=0.04
nucleus, p=0.26
cytoplasm, p=0.24

===== ACKR2_RAT ====
membrane, p=0.96
cytoplasm, p=0.00
mitochondrion, p=0.03
nucleus, p=0.00
cytoplasm, p=0.00

===== TOM5_YEAST ====
membrane, p=0.38
cytoplasm, p=0.18
mitochondrion, p=0.05
nucleus, 